# Chapter 8: Planning and Learning with Tabular Methods

**Chapter Overview**: This chapter unifies model-based methods (like dynamic programming, which use planning with an environment model) and model-free methods (like Monte Carlo and TD learning, which rely directly on experience). It introduces integrated architectures like Dyna that combine learning and planning. Key themes include using models to simulate experience, the trade-offs between expected and sample updates, focusing backups efficiently, and decision-time planning techniques like MCTS. As the capstone of Part I (tabular methods), it recaps core dimensions of RL methods and bridges to Part II's focus on scaling via function approximation.

**Key Unification Insight**: All RL methods compute value functions via backups (looking ahead to future events). Planning uses simulated experience from a model; learning uses real experience. This allows seamless integration: apply learning algorithms to model-generated data for planning.

**Sidenote on Planning vs. Learning**: Planning is "deliberative" (model-based foresight), learning is "reactive" (experience-based adaptation). But they're not opposites—many methods blend them. Intuitively, planning is like mentally rehearsing scenarios; learning is trial-and-error in the real world.

---

## 8.1 Models and Planning

**Models**: Anything an agent uses to predict environment responses to actions.

- **Given**: State $s$, action $a$
- **Predicts**: Next state $s'$ and reward $r$

**Types of Models**:
- **Distribution model**: Gives all possible $s', r$ and their probabilities (e.g., DP's $p(s', r|s, a)$ from Equation 3.2).
- **Sample model**: Samples one possible $s', r$ according to probabilities (e.g., blackjack simulation in Chapter 5).

**Intuition**: Distribution models are "stronger" (can generate samples and probabilities) but harder to obtain. Sample models are easier (e.g., simulate dice rolls) but may introduce sampling variance.

**Sidenote**: Models simulate experience. From a state and policy, generate episodes (trajectories). This "simulated experience" can train RL methods just like real experience.

**Planning**: Computation to improve policy/value function using a model.
- **State-space planning**: Focus of RL (vs. plan-space planning like evolutionary methods).
- **Core Structure**: Compute values via backups on simulated experience.

**Diagram Insight**: All state-space planning fits: (1) Compute values as intermediate step; (2) Use backups on simulated experience.

![Common Structure of Planning and Learning](../img/fig8_1.png)

**Unification with Learning**: Replace real experience in learning methods (e.g., TD) with model-simulated experience for planning. Both estimate values via backups; difference is experience source.

---

## 8.2 Dyna: Integrated Planning, Acting, and Learning

**Dyna Architecture**: Online agent integrating planning, acting, model-learning, and direct RL (all in parallel).

- **Direct RL**: Improve value/policy from real experience (e.g., Q-learning).
- **Model-learning**: Update model from real experience.
- **Planning**: Indirect RL—improve value/policy from simulated experience.

**Diagram**:

![Dyna Architecture](../img/fig8_1.png)  <!-- Note: This is Figure 8.1 from the text -->

**Relationships** (see diagram on page 163):
- Experience → Model (model-learning)
- Experience → Value/Policy (direct RL)
- Model → Simulated Experience → Value/Policy (planning)

**Search Control**: Process selecting starting states/actions for simulated experiences (e.g., random from experienced pairs).

**Tabular Dyna-Q Algorithm** (pseudocode on page 164):
- Initialize $Q(s,a)$ and Model$(s,a)$
- Loop forever:
  - (a) $S$ ← current state
  - (b) $A$ ← ε-greedy$(S, Q)$
  - (c) Execute $A$, observe $R, S'$
  - (d) $Q(S,A) \leftarrow Q(S,A) + \alpha [R + \gamma \max_{a'} Q(S',a') - Q(S,A)]$  (direct RL, one-step tabular Q-learning)
  - (e) Model$(S,A) \leftarrow R, S'$  (assuming deterministic env)
  - (f) For $n$ planning steps: Random experienced $S_{rand}, A_{rand}$; Simulate $R, S'$ from Model; Update $Q$ as in (d)

**Key Parameters**:
- $n$: Planning steps per real step (balances computation).
- Assumes deterministic model; queries only experienced pairs.

**Example: Dyna Maze** (Figure 8.2):
- Task: From S to G quickly.
- Dyna-Q with more $n$ learns faster (e.g., $n=50$ builds extensive policy in one episode).

![Dyna Maze Learning Curves](../img/fig8_2.png)

![Policies During Learning](../img/fig8_3.png)

**Intuition**: Planning builds "mental models" to accelerate learning. Without planning ($n=0$), policy grows one step per episode; with planning, it expands rapidly via backups.

**Sidenote**: Dyna-Q unifies: Same Q-update for real/simulated experience. Agent is always acting, planning, learning—model improves as experience grows.

**Exercise Insight**: Multi-step methods (Ch. 7) might match Dyna if model accurate, but Dyna's one-step simplicity + many simulations often wins.

---

## 8.3 When the Model Is Wrong

**Issue**: Models can be inaccurate (initially empty or environment changes).

**Example: Blocking Maze** (Figure 8.4):
- Path blocks after 1000 steps; new path opens.
- Dyna-Q recovers but wanders initially.

**Worse Case: Shortcut Maze** (Figure 8.5):
- Shortcut opens; Dyna-Q misses it (model says impossible, ε-greedy rarely explores enough).

**Solution: Dyna-Q+**:
- Track time $τ$ since last real try of $(s,a)$.
- Add exploration bonus: Simulated reward $r + κ\sqrt{τ}$ (encourages testing stale transitions).
- Solves shortcut by exploring long-untried actions.

![Blocking Maze Performance](../img/fig8_4.png)

![Shortcut Maze Performance](../img/fig8_5.png)

**Intuition**: Wrong models cause "model traps." Add curiosity (bonus for uncertainty) to detect changes.

**Sidenote**: Stochastic envs: Average model over experiences. Changing envs: Weight recent experiences more.

---

## 8.4 Prioritized Sweeping

**Issue**: Uniform random planning (Dyna-Q) wastes updates on low-impact states.

**Solution**: Focus backups backward from high-change states (like reverse BFS).

**Prioritized Sweeping Algorithm** (for deterministic env, pseudocode on page 170):
- Use priority queue: State-action pairs prioritized by update magnitude $P = |R + \gamma \max_a Q(S',a) - Q(S,A)|$.
- If $P > θ$ (small threshold), insert/update queue.
- Update top pair; propagate changes to predecessors.

**Intuition**: Chain reactions—big changes in one state affect predecessors. Queue ensures efficient backward focus.

**Advantages**:
- Faster than uniform (e.g., maze: 3x fewer backups for same performance).
- Stochastic extension: Sample successors.

**Example: Rod Maneuvering** (Figure 8.6): Solves large (14k states) deterministic task efficiently.

![Prioritized Sweeping on Maze](../img/fig8_6.png)  <!-- Assuming this is the rod figure or maze comparison -->

**Sidenote**: Like heuristic search but retains values long-term. "Small backups" (van Seijen & Sutton, 2013) take this further: Probability-weighted single transitions without sampling.

---

## 8.5 Expected vs. Sample Updates

**Dimensions of Updates**:
- State vs. action values.
- Optimal vs. arbitrary policy.
- Expected (all successors) vs. sample (one successor).

**Trade-off**: Expected: Accurate but compute-heavy (branching factor $b$). Sample: Noisy but cheap.

**Efficiency Insight** (Figure 8.7): Sample often better for large $b$ (reduces error faster per computation).

![Expected vs Sample Efficiency](../img/fig8_7.png)

**General Rule**: Sample wins for large $b$, deep backups; expected for small $b$, shallow.

**Sidenote**: Planning favors samples (easy from model); real experience favors expected (to reduce variance).

---

## 8.6 Trajectory Sampling

**Update Distribution**: Where to focus backups?
- **Uniform**: All states equally (like DP sweeps)—wastes on irrelevant states.
- **On-policy**: Sample trajectories following current policy—focuses on likely states.

**Advantages of On-Policy** (Figure 8.8): Faster initial learning (relevant states); but uniform may win long-term.

![Uniform vs On-Policy](../img/fig8_8.png)

**Intuition**: Like importance sampling—prioritize states under current behavior.

---

## 8.7 Real-time Dynamic Programming (RTDP)

**RTDP**: Asynchronous value iteration with on-policy trajectory sampling.
- Update only visited states greedily.
- Converges to optimal (with exploring starts).

**Advantages**: Ignores irrelevant states; policy improves during computation.

**Example: Racetrack** (Exercise 5.12): RTDP updates fewer states, finds near-optimal policy faster than full sweeps.

**Intuition**: "Relevant states" are reachable under optimal policy—RTDP naturally focuses there without visiting all.

**Sidenote**: Partial policies (optimal only on relevant states) suffice; RTDP guarantees this without infinite visits everywhere.

---

## 8.8 Planning at Decision Time

**Background Planning**: Improve overall policy/value (e.g., Dyna)—unfocused.
- **Decision-time Planning**: From current state, search deeply to select one action (discard backups after).

**Intuition**: Focus computation on now; useful when fast responses aren't needed (e.g., games).

---

## 8.9 Heuristic Search

**Classical AI Method**: From current state (root), build search tree; evaluate leaves with heuristic $v$; backup maxes to root; pick best action.

**RL Twist**: Improve heuristic $v$ over time via backups—blends with learning.

![Heuristic Search as Backups](../img/fig8_9.png)

**Intuition**: Deep search = many small backups focused downstream. Better than unfocused for current decision.

---

## 8.10 Rollout Algorithms

**Method**: Monte Carlo control from current state: Simulate trajectories per action using rollout policy $\pi$; average returns for $q(s,a)$; pick $\arg\max_a$.

**No Storage**: Discard estimates after action choice.

**Improvement**: Policy iteration—rollouts improve over $\pi$ (if $\pi$ good, rollouts better).

**Intuition**: Like MC but decision-time; balances trials vs. depth (truncate + bootstrap if needed).

**Sidenote**: Parallelizable; prune poor actions. Enhances weak $\pi$ via lookahead.

---

## 8.11 Monte Carlo Tree Search (MCTS)

**Method**: Build asymmetric tree from current state via iterations:
1. **Selection**: Tree policy (e.g., UCB) to leaf.
2. **Expansion**: Add child(ren).
3. **Simulation**: Rollout policy to end; get return.
4. **Backup**: Update $Q$ along path (average returns).

Pick action with best root $Q$.

![MCTS Process](../img/fig8_10.png)

**Intuition**: Tree focuses on promising branches; rollouts explore beyond. Balances exploration/exploitation via tree policy.

**RL View**: MC control with partial $q$ table (tree); discards between moves.

**Sidenote**: AlphaGo (Ch. 16) extends with neural nets. MCTS revolutionized AI games by smart sampling.

---

## 8.12 Summary of the Chapter

**Recap**: Models enable planning (simulated backups). Dyna integrates all; prioritized/trajectory sampling focuses efficiently. Expected/sample, background/decision-time are key axes. Rollout/MCTS: Powerful decision-time methods.

**Bridge to Part II**: Part I assumed tabular (exact values per state)—limits to small problems. Part II scales via function approximation: Approximate values over large spaces, enabling real-world RL.

---

## 8.13 Summary of Part I: Dimensions

**Core Dimensions** (Figure 8.11):
- **Depth**: Shallow (one-step) to deep (full episodes).
- **Width**: Expected (all branches) to sample (one branch).
- **On/Off-Policy**: Learn current vs. optimal policy.

![Dimensions of RL Methods](../img/fig8_11.png)

**Other Dimensions**:
- Episodic/continuing, discounted/undiscounted.
- State/action/afterstate values.
- Exploration: ε-greedy, optimistic, UCB.
- Synchronous/asynchronous.
- Real/simulated experience.
- Update location/timing/memory.

**Intuition**: Vast design space—mix for task. Function approximation (Part II) adds orthogonal scaling dimension.

**Part I Takeaway**: Tabular methods build foundations; understand backups, models, integration for intuition.

---

## Exercises

### Exercise 8.1 Solution
**Q**: Why might multi-step bootstrapping match Dyna?

**A**: Multi-step uses real multi-transitions; Dyna simulates many one-steps. If model accurate, similar. But Dyna flexible (arbitrary simulations); multi-step limited by real data—Dyna often better for planning.

### Exercise 8.2 Solution
**Q**: Why does nonplanning look poor in Fig 8.3?

**A**: It's one-step. Multi-step could propagate faster, potentially matching Dyna. But Dyna's simulations allow more flexible/backward focus.

### Exercise 8.3 Solution
**Q**: Alternate Dyna-Q+: Use $\sqrt{\tau}$ in action selection, not updates.

**A**: 
- **Strength**: Direct exploration in real actions.
- **Weakness**: Less in planning (may miss long chains). Gridworld: Good for simple changes; worse for deep dependencies.

### Exercise 8.4 Solution
**Q**: Modify Dyna-Q for stochastic envs; issues in changing envs; fix.

**A**: Model averages experiences (e.g., count-based probabilities). Changing envs: Forgets slowly—use recency weighting (e.g., higher α for model updates) or detect changes/reset.

### Exercise 8.5 Solution
**Q**: Prioritized sweeping on rod maneuvering.

**A**: (Descriptive; see text example—efficient for large deterministic tasks by focusing backups.)

(Note: Some exercises are thought experiments; solutions are conceptual.)

---
---